### Dataset Utilities for Algo-Fin Data

In [ ]:
import pickle
import pandas as pd

In [ ]:
import torch
import numpy as np
from torch.utils.data import Dataset
import sklearn.datasets as skds
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
import math
from IPython import display
from time import sleep
import pickle
import pandas as pd

In [ ]:
import import_ipynb
from feeds import DataFeed, BackFeed, clean_feed_nulls
from featfuncs import USE_COLS_DICT_ORIG,update_use_cols_dict 
from featfuncs import add_addl_features_feed,add_sym_feature_feed,add_logical_features_feed
from featfuncs import DiscretizeK,discretize_features_feed
from utils import MyDS

In [ ]:
from sklearn.preprocessing import KBinsDiscretizer
from datetime import datetime

In [ ]:
from dsutils_algofin import create_algofin_data

In [ ]:
DATAPATH='/Users/a112956/MyCode/algo_fin_root/temp_data/'
MODELPATH='./saved_models/'
DATAFEED=False
VALFEED=False
SYNTHETIC=True
simple=True
sigma=0.01
use_cols='alllog'
LOADFEED=False
nd,nw=5,30
PLOTFEED=False

In [ ]:
if LOADFEED and DATAFEED and not SYNTHETIC: 
    with open(DATAPATH+'btdatafeed.pickle','rb') as f: datafeed=pickle.load(f)
if LOADFEED and VALFEED and not SYNTHETIC: 
    with open(DATAPATH+'btfeed.pickle','rb') as f: feed=pickle.load(f)
if LOADFEED and SYNTHETIC:
    feedname='syn_synthetic_'+str(simple)+'.pickle'
    with open(DATAPATH+feedname,'rb') as f: feed=pickle.load(f)
    datafeed=feed

## Generate Feed

In [ ]:
if not LOADFEED and SYNTHETIC:
    print('Creating feed')
    feed=BackFeed(tickers=None,nd=nd,nw=nw,interval='5m',synthetic=True,simple=simple,sigma=sigma)
    print('Processing feed')
    add_addl_features_feed(feed,tickers=feed.tickers)
    add_sym_feature_feed(feed,tickers=feed.tickers)
    add_logical_features_feed(feed)

## Plot Feed

In [ ]:
if SYNTHETIC and PLOTFEED:
    _=[feed.plot_ticker_date('SYN',d,show_prev=True) for d in feed.ndata['SYN']]

## Data Generation

In [ ]:
clean_feed_nulls(feed)

In [ ]:
if not SYNTHETIC: 
    with open(MODELPATH+'discretizers.pickle','rb') as f: discretizers=pickle.load(f)

In [ ]:
if not SYNTHETIC: DkT5,DkT10,DkD=discretizers[0],discretizers[1],discretizers[2]

In [ ]:
if DATAFEED: feed=datafeed
if SYNTHETIC or DATAFEED:
    df_train,ds_train,train_loader,df_test,ds_test,test_loader,DkT5,DkT10,DkD=create_algofin_data(feed=feed,
                                                                            cols='alllog',
                                                                            discretize=True,
                                                                            SYN=SYNTHETIC)

In [ ]:
if not SYNTHETIC:
    df_val,ds_val,val_loader,df_val_test,ds_val_test,val_test_loader,DkT5,DkT10,DkD=create_algofin_data(feed=feed,
                                                                            cols='alllog',
                                                                            discretize=True,
                                                                            estgiven=True,
                                                                            DkT5=DkT5,DkT10=DkT10,
                                                                            DkD=DkD,SYN=SYNTHETIC)

### SAVE & USE data

df_train,df_test etc

In [ ]:
# clean_feed_nulls(feed)
# MODELPATH='./saved_models/'
# with open(MODELPATH+'discretizers.pickle','rb') as f: discretizers=pickle.load(f)
# discretize_features_feed(datafeed,discretizers[2],use_cols)

In [ ]:
# # SAVE_PATH='~/DataLocal/fin_regression_summer_proj/'
# df_train.to_csv(SAVE_PATH+'df_train.csv',index=False)
# df_test.to_csv(SAVE_PATH+'df_test.csv',index=False)
# df_val.to_csv(SAVE_PATH+'df_val.csv',index=False)
# df_val_test.to_csv(SAVE_PATH+'df_val_test.csv',index=False)

In [ ]:
# # for SYNTHETIC data
# SAVE_PATH='~/DataLocal/fin_regression_summer_proj/'
# df_train.to_csv(SAVE_PATH+'df_train_'+'syn_'+str(simple)+'.csv',index=False)
# df_test.to_csv(SAVE_PATH+'df_test_'+'syn_'+str(simple)+'.csv',index=False)